In [1]:
%load_ext autoreload
%autoreload 2

## Install dependencies

In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git --no-deps
!pip install git+https://github.com/murtylab/neuroai-notebooks.git --upgrade
!mkdir datasets

## Load the model and set up a hook

In [2]:
import torch
from neuroai.models import CLIPRN50

backbone_model = CLIPRN50(pretrained=True, download_root="./pretrained_checkpoints")

## Extracting features

In [3]:
from neuroai.utils import ForwardHook

hook = ForwardHook(
    model=backbone_model,
    hook_layer_name = "model.layer3.1.conv1" # << extract activations from this layer
)

In [ ]:
x = torch.randn(1,3,224,224)
y = backbone_model(x)

## it's of shape (batch, channels, height, width)
hook.output.shape

## Downloading the NSD dataset

In [5]:
from neuroai.datasets import download_and_extract_nsd

download_and_extract_nsd(zip_filename = "nsd.zip", output_folder = "./datasets")

In [ ]:
from neuroai.datasets import NSDAllSubjectSingleRegion
import torch.nn as nn

# device = "cuda:0" if torch.cuda.is_available() else "cpu"

device = "mps:0"
print(f"Using device: {device}")
## lets move the model to our device
backbone_model = backbone_model.to(device)

region = "EBA"
dataset = NSDAllSubjectSingleRegion(
    folder="./datasets/nsd",
    region=region,
    transforms=backbone_model.transforms,
    subset="train",
    train_test_split=0.8
)

## Collecting image features and the corresponding voxels 

In [14]:
from tqdm import tqdm

def collect_features_and_voxels(
    dataset: NSDAllSubjectSingleRegion,
    backbone_model: nn.Module,
    hook: ForwardHook,
    device: str,
    subject_id: str
):

    ## this is where we dump our data
    all_features = []
    all_fmri_voxels = []
    backbone_model = backbone_model.to(device)

    with torch.no_grad():
        for i in tqdm(range(len(dataset)), desc="Collecting features and voxels"):
            image: torch.Tensor = dataset[i]["image"]

            ## (voxels) -> (1, voxels)
            fmri_data: torch.Tensor = dataset[i]["fmri_response"][subject_id].unsqueeze(0)

            ## (channels, height, width) -> (1, channels, height, width)
            image = image.unsqueeze(0)
            image = image.to(device)
            y = backbone_model(image)

            ## making sure that we're moving stuff back to the RAM with .cpu()
            hook_output = hook.output.cpu()
            all_features.append(hook_output)
            all_fmri_voxels.append(fmri_data.cpu())

    return {
        "features": torch.cat(all_features, dim=0),
        "voxels": torch.cat(all_fmri_voxels, dim=0)
    }

In [ ]:
train_subject_id = "s2"

data = collect_features_and_voxels(
    dataset=dataset,
    backbone_model=backbone_model,
    hook=hook,
    device=device,
    subject_id=train_subject_id
)
## num samples, channels, height, width
print(f"Shape of features", data["features"].shape)

## num samples, voxels
print(f"Shape of voxels", data["voxels"].shape)

## Training a model

In [40]:
from neuroai.utils.regression import ridge_regression
from neuroai.utils.regression import RidgeModel

ridge_result = ridge_regression(
    x_train=data["features"],
    y_train=data["voxels"],
    flatten_x=True, ## this squishes features to be (batch, channels*height*width)
    predict_mean=False ## set this to True to predict the mean response of a region
)

model = RidgeModel(
    backbone_model=backbone_model,
    transforms=backbone_model.transforms,
    hook_layer_name="model.layer3.1.conv1", ## make sure that this is the same as the one used in ForwardHook
    ridge_result=ridge_result,
)

## Now let's evaluate on data from another subject

In [ ]:
valid_subject_ids = ["s1", "s2", "s5", "s7"]

eval_subject_id = "s1"

dataset = NSDAllSubjectSingleRegion(
    folder="./datasets/nsd",
    region=region,
    transforms=backbone_model.transforms,
    subset="test", ## << make sure that this is set to "test" when evaluating
    train_test_split=0.8
)

data = collect_features_and_voxels(
    dataset=dataset,
    backbone_model=backbone_model,
    hook=hook,
    device=device,
    subject_id=eval_subject_id
)

correlation = model.evaluate(
    x_test=data["features"],
    y_test=data["voxels"],
    predict_mean=True
)
print(f"Region: {region} Trained on subject: {train_subject_id} Eval on subject: {eval_subject_id}: {correlation}")

## Running the model on your images

In [26]:
# !wget -O cat.jpg "https://plus.unsplash.com/premium_photo-1667030474693-6d0632f97029"
!wget -O castle.jpg "https://upload.wikimedia.org/wikipedia/commons/thumb/4/40/Panorámica_Otoño_Alcázar_de_Segovia.jpg/1200px-Panorámica_Otoño_Alcázar_de_Segovia.jpg"
!wget -O monkey.jpg "https://images.unsplash.com/photo-1581828060707-37894f1ed9b8"

In [23]:
from PIL import Image

images = [
        Image.open("monkey.jpg"),
        Image.open("castle.jpg"),
    ]

results = model.run(
    images = images
)
print(results)


## Bonus: Feature Visualization

In [58]:
# !pip install torch-dreams==4.0.0

In [ ]:
import matplotlib.pyplot as plt
import torchvision.models as models
from torch_dreams import Dreamer

dreamy_boi = Dreamer(model, device = device, quiet=False)


def maximize_voxel(layer_outputs):
    loss = layer_outputs[0].mean()
    return -loss

image_param = dreamy_boi.render(
    layers = [model],
    width =224,
    height =224,
    custom_func = maximize_voxel,
    scale_min=1.0,
    scale_max=1.0,
    iters=300,
    translate_x=0.5,
    translate_y=0.5,
    rotate_degrees=5
)

plt.imshow(image_param)
plt.show()

In [55]:
image_param.save("visualization_eba.png")

